In [6]:
# seer_minimal.py  –  works with the real SEER dataset
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


csv_path = "/kaggle/input/seer-breast-cancer-data/SEER Breast Cancer Dataset .csv"

# 2. load ----------------------------------------------------------------------
df = pd.read_csv(csv_path)

In [11]:
df.dtypes

Age                         int64
Race                       object
Marital Status             object
Unnamed: 3                float64
T Stage                    object
N Stage                    object
6th Stage                  object
Grade                      object
A Stage                    object
Tumor Size                  int64
Estrogen Status            object
Progesterone Status        object
Regional Node Examined      int64
Reginol Node Positive       int64
Survival Months             int64
Status                     object
dtype: object

In [13]:
df.drop(columns=["Unnamed: 3"], inplace=True)

In [16]:
df.shape

(4024, 15)

In [21]:
# fix column names
df.columns = df.columns.str.strip()

In [22]:


# 2. basic cleaning -----------------------------------------------------------
df = df.dropna(subset=["Status"])
y = df["Status"].map({"Dead": 1, "Alive": 0}).astype(int)
X = df.drop(columns=["Status"])

# 3. column types -------------------------------------------------------------
num_cols = ["Age", "Tumor Size", "Regional Node Examined",
            "Reginol Node Positive", "Survival Months"]

cat_cols = ["Race", "Marital Status", "T Stage", "N Stage", "6th Stage",
            "Grade", "A Stage", "Estrogen Status", "Progesterone Status"]

# 4. pre-processing -----------------------------------------------------------
numeric_pipe   = Pipeline([("scaler", StandardScaler())])
categorical_pipe = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])

pre = ColumnTransformer(
        [("num", numeric_pipe, num_cols),
         ("cat", categorical_pipe, cat_cols)])

# 5. train/test split ---------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42, stratify=y)

# 6. models -------------------------------------------------------------------
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "RandomForest": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGBoost": XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=4,
                             subsample=0.8, colsample_bytree=0.8, eval_metric="logloss",
                             random_state=42)
}

# 7. train & evaluate ---------------------------------------------------------
for name, model in models.items():
    pipe = Pipeline([("prep", pre), ("clf", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    print(f"\n{name}  –  test metrics")
    print("Accuracy :", round(accuracy_score(y_test, preds), 3))
    print("Precision:", round(precision_score(y_test, preds), 3))
    print("Recall   :", round(recall_score(y_test, preds), 3))
    print("F1-score :", round(f1_score(y_test, preds), 3))


LogisticRegression  –  test metrics
Accuracy : 0.806
Precision: 0.421
Recall   : 0.715
F1-score : 0.53

RandomForest  –  test metrics
Accuracy : 0.896
Precision: 0.81
Recall   : 0.415
F1-score : 0.548

XGBoost  –  test metrics
Accuracy : 0.916
Precision: 0.867
Recall   : 0.528
F1-score : 0.657
